In [49]:
import numpy as np
import string

np.random.seed(1234)

In [50]:
initial = {} # start of a phrase
first_order = {} # second word only
second_order = {}

In [51]:
def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [52]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘robert_frost.txt’ already there; not retrieving.



In [53]:
def add2dict(d, k, v):
  if k not in d:
    d[k] = {}
  d[k][v] = d[k].get(v,0) + 1

# [cat, cat, dog, dog, dog, dog, dog, mouse, ...]

In [54]:
for line in open('robert_frost.txt'):
  tokens = remove_punctuation(line.rstrip().lower()).split()

  T = len(tokens)
  for i in range(T):
    t = tokens[i]
    if i == 0:
      # measure the distribution of the first word
      initial[t] = initial.get(t, 0.) + 1
    else:
      t_1 = tokens[i-1]
      if i == T - 1:
        # measure probability of ending the line
        add2dict(second_order, (t_1, t), 'END')
      if i == 1:
        # measure distribution of second word
        # given only first word
        add2dict(first_order, t_1, t)
      else:
        t_2 = tokens[i-2]
        add2dict(second_order, (t_2, t_1), t)

In [55]:
# normalize the distributions
initial_total = sum(initial.values())
for t, c in initial.items():
    initial[t] = c / initial_total

In [56]:
for key,maps in first_order.items():
  sums = sum(maps.values())
  for k,v in maps.items():
    maps[k] = v/sums

In [57]:
for key,maps in second_order.items():
  sums = sum(maps.values())
  for k,v in maps.items():
    maps[k] = v/sums

In [58]:
def sample_word(d):
  # print "d:", d
  p0 = np.random.random()
  # print "p0:", p0
  cumulative = 0
  for t, p in d.items():
    cumulative += p
    if p0 < cumulative:
      return t
  assert(False) # should never get here

In [59]:
def generate():
  for i in range(4): # generate 4 lines
    sentence = []

    # initial word
    w0 = sample_word(initial)
    sentence.append(w0)

    # sample second word
    w1 = sample_word(first_order[w0])
    sentence.append(w1)

    # second-order transitions until END
    while True:
      w2 = sample_word(second_order[(w0, w1)])
      if w2 == 'END':
        break
      sentence.append(w2)
      w0 = w1
      w1 = w2
    print(' '.join(sentence))

In [60]:
generate()

i went to bed alone and left me
might just as empty
but it isnt as if and thats not all the money goes so fast
you couldnt call it living for it aint


In [61]:
len(initial)

305

In [62]:
count = 0
for k,v in first_order.items():
  count+=len(v)
count

1195

In [63]:
count = 0
for k,v in second_order.items():
  count+=len(v)
count

8854

In [64]:
# Exercise:
#
# Determine the vocabulary size (V)
# MY COMMENT: 305
# We know that pi has shape V, A1 has shape V x V, and A2 has shape V x V x V
# MY COMMENT: A1 has 1195 values and A2 has 8854 values. V^3 = 305^3 = 28372625
# So we store 0.036% values compared to tensor with mostly zeros
# In comparison, how many values are stored in our dictionaries?

In [65]:
# Exercise 2:
# We can skip the step where we accumulate all the possible next words in a list
# E.g. [cat, cat, dog, dog, dog, ...]
#
# Instead, like we do with the initial state distribution, create the dictionary
# of counts directly as you loop through the data.
#
# You'll no longer need list2pdict()